In [ ]:
!pip install geemap --quiet
!pip install earthengine-api --quiet
!pip install PyCRS --quiet
!pip install restee --quiet
!pip install config --quiet
!pip install geojson --quiet

import geemap, ee, os, sys, pycrs, datetime
import geemap.colormaps as cm

from ipyleaflet import *
from ipywidgets import Label
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.dates as mdates
import matplotlib.ticker as ticker

import config
import json
import csv
import geojson
import geopandas as gpd
import rasterio
import importlib
import numpy  as np

import requests
import zipfile
import os
from os import listdir
from os.path import isfile,join

In [ ]:
# ADJUST TO YOUR CREDENTIALS

ee.Authenticate()
ee.Initialize(project='ee-yourproject')

In [ ]:
# ADJUST TO YOUR PATH

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!cp /content/drive/MyDrive/GPPmodels/CRO_model_GPP.py .
import CRO_model_GPP as model_GPP_CRO

!cp /content/drive/MyDrive/GPPmodels/CSH_model_GPP.py .
import CSH_model_GPP as model_GPP_CSH

!cp /content/drive/MyDrive/GPPmodels/DBF_model_GPP.py .
import DBF_model_GPP as model_GPP_DBF

!cp /content/drive/MyDrive/GPPmodels/EBF_model_GPP.py .
import EBF_model_GPP as model_GPP_EBF

!cp /content/drive/MyDrive/GPPmodels/ENF_model_GPP.py .
import ENF_model_GPP as model_GPP_ENF

!cp /content/drive/MyDrive/GPPmodels/GRA_model_GPP.py .
import GRA_model_GPP as model_GPP_GRA

!cp /content/drive/MyDrive/GPPmodels/MF_model_GPP.py .
import MF_model_GPP as model_GPP_MF

!cp /content/drive/MyDrive/GPPmodels/OSH_model_GPP.py .
import OSH_model_GPP as model_GPP_OSH

!cp /content/drive/MyDrive/GPPmodels/SAV_model_GPP.py .
import SAV_model_GPP as model_GPP_SAV

!cp /content/drive/MyDrive/GPPmodels/WET_model_GPP.py .
import WET_model_GPP as model_GPP_WET

Mounted at /content/drive


In [ ]:
# ADJUST TO YOUR REGION OF INTEREST

# Example polygon
DBF_CZ_Stn_100m = ee.Geometry.Polygon([[[17.968539226026834, 49.03583854094053],[17.96860896346122, 49.03568028280741],[17.968700158567728, 49.035543125351474],[17.968818175764383, 49.035423551876185],
          [17.968968379469217, 49.03531101187216],[17.9691454052642, 49.035223089816846],[17.96935461756736, 49.035152752060746],[17.96964429614097, 49.03509648178424],[17.96992324587852, 49.035078897309766],
          [17.970207560034098, 49.03509648178424],[17.970475780935587, 49.0351633027305],[17.970706450910868, 49.035247708007994],[17.970910298796, 49.03537079878099],[17.971081960172953, 49.03550795671209],
          [17.971216070623697, 49.035697867069295],[17.971269714803995, 49.03586667566702],[17.971269714803995, 49.03605306782822],[17.971226799459757, 49.03624297610432],[17.971098053427042, 49.036429366855465],
          [17.97091566321403, 49.0365841058164],[17.970722544164957, 49.036703676501965],[17.970481145353617, 49.036791595940336],[17.97018610236198, 49.03685489783976],[17.96989642378837, 49.036883031991415],
          [17.969590651960672, 49.03684786429935],[17.969349253149332, 49.036795112714636],[17.969102489919962, 49.03671774362256],[17.96890937087089, 49.03660168975877],[17.968759167166056, 49.036475085234926],
          [17.96864651438743, 49.036344963582984],[17.968560683698954, 49.036186707061056],[17.968523132772745, 49.035989764908386]]])

In [ ]:
########################################################################
# ADJUST

# SELECT TIME FRAME
startDateStr = '2017-03-21'
endDateStr = '2021-01-01'
timeWindows = 1

# SELECT REGION
roi = DBF_CZ_Stn_100m

# SELECT MODEL
variable = 'GPP_DBF'

# SET EXPORT PATH
assetPath='projects/ee-declerckem3/assets/GPR-S2-GPP/MF-CZ-Lnz-100m/'

########################################################################


In [ ]:
# Set some variables
S2MSI = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
scaleFactor_S2 = 0.0001
variables_GREEN = {variable:[variable, 35, 0.001]}
sys.setrecursionlimit(10000000)


# Clouds mask
def maskS2cloud(image):
    scl = image.select('SCL')
    mask = scl.neq(3).And(  # 3 = Cloud shadows
           scl.neq(8).And(  # 8 = Cloud medium probability
           scl.neq(9).And(  # 9 = Cloud high probability
           scl.neq(10))))   # 10 = Thin cirrus
    return image.updateMask(mask).divide(10000).copyProperties(image, image.propertyNames())

def apply_snow_mask(image):
    # Calculate NDSI
    ndsi = image.normalizedDifference(['B3', 'B11']).rename('NDSI')
    snow_mask = ndsi.lt(0.4)
    return image.updateMask(snow_mask)

# Bare cover and water mask
# bare_cover = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('bare-coverfraction').lte(90);
# lakes = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification').eq(80)
# lakemask = lakes.eq(0);


# Some auxilary functions
def sequence_GREEN(variable):
	sequence_GREEN = []
	model = globals()['model_' + variable]
	for i in range(0, model.XTrain_dim_GREEN):
		sequence_GREEN.append(str(i))
	return sequence_GREEN

def getInputDates(i, startDateGEE):
    fecha_inicio = startDateGEE.advance(ee.Number(i).multiply(timeWindows), 'day')
    fecha_fin = fecha_inicio.advance(timeWindows, 'day')
    fecha_str = datetime.datetime.utcfromtimestamp(fecha_inicio.getInfo()['value'] / 1000.0).strftime('%Y%m%d')
    return {'fecha_inicio': fecha_inicio, 'fecha_fin': fecha_fin, 'fecha_str': fecha_str}

def addVariables(image):
  date = ee.Date(image.get("system:time_start"));
  years = date.difference(ee.Date('1970-01-01'),'days');
  return image.addBands(ee.Image(years).rename('t').float());


# Gaussian processes formulation
def calculate_GREEN(fecha_inicio, fecha_fin, variable):

  model = globals()['model_' + variable]

  image = ee.Image(S2MSI
  .filterDate(fecha_inicio, fecha_fin)
  .filterBounds(roi)
  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20)
  .map(maskS2cloud)
  .map(apply_snow_mask)
  .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])
  .mean()
  # .multiply(scaleFactor_S2) # comment out if already integrated in the cloud mask
  )

  im_norm_ell2D_hypell = image.subtract(model.mx_GREEN).divide(model.sx_GREEN).multiply(model.hyp_ell_GREEN).toArray().toArray(1);
  im_norm_ell2D = image.subtract(model.mx_GREEN).divide(model.sx_GREEN).toArray().toArray(1);
  PtTPt  = im_norm_ell2D_hypell.matrixTranspose().matrixMultiply(im_norm_ell2D).arrayProject([0]).multiply(-0.5); #OK
  PtTDX  = ee.Image(model.X_train_GREEN).matrixMultiply(im_norm_ell2D_hypell).arrayProject([0]).arrayFlatten([sequence_GREEN(variable)]);
  arg1   = PtTPt.exp().multiply(model.hyp_sig_GREEN);
  k_star = PtTDX.subtract(model.XDX_pre_calc_GREEN.multiply(0.5)).exp().toArray();
  mean_pred = k_star.arrayDotProduct(model.alpha_coefficients_GREEN.toArray()).multiply(arg1);
  mean_pred = mean_pred.toArray(1).arrayProject([0]).arrayFlatten([[variable]]);
  mean_pred = mean_pred.add(model.mean_model_GREEN);
  k_star_uncert = PtTDX.subtract(model.XDX_pre_calc_GREEN.multiply(0.5)).exp().multiply(arg1).toArray();
  Vvector = ee.Image(model.Linv_pre_calc_GREEN).matrixMultiply(k_star_uncert.toArray(0).toArray(1)).arrayProject([0])
  Variance = ee.Image(model.hyp_sig_unc_GREEN).toArray().subtract(Vvector.arrayDotProduct(Vvector)).abs().sqrt()
  Variance = Variance.toArray(1).arrayProject([0]).arrayFlatten([[variable + '_UNC']])

  image= image.addBands(mean_pred);
  image = image.addBands(Variance);

  return image.select(variable, variable + '_UNC')


#Iterations:
def maploop():
    startDate = datetime.datetime.strptime(startDateStr, "%Y-%m-%d").date()
    endDate = datetime.datetime.strptime(endDateStr, "%Y-%m-%d").date()
    daysIterations = (endDate - startDate).days // timeWindows
    startDateGEE = ee.Date(startDateStr)

    # print(f"Start: {startDate}, End: {endDate}, Iterations: {daysIterations}")

    for i in range(0, daysIterations):
        inputDates = getInputDates(i, startDateGEE)
        fecha_inicio = inputDates['fecha_inicio']
        fecha_fin = inputDates['fecha_fin']
        fecha_str = inputDates['fecha_str']

        # print(f"\n Exporting for {fecha_str}...")

        imageHolder = ee.Image().set('system:time_start', fecha_inicio.millis())

        for variable_GREEN in variables_GREEN:
            params = variables_GREEN[variable_GREEN]
            variable = params[0]

            imagen = calculate_GREEN(fecha_inicio, fecha_fin, variable)
            image_export = imageHolder.addBands(imagen).select([variable, variable + '_UNC'])
            image_export = image_export.set('system:time_start', fecha_inicio.millis())

            exportar = ee.batch.Export.image.toAsset(
                assetId=assetPath + fecha_str + variable_GREEN,
                image=image_export,
                maxPixels=4731453308,
                description=fecha_str + variable_GREEN,
                scale=20,
                region=roi
            )

            exportar.start()


maploop()